In [1]:
import tkinter as tk
from tkinter import ttk
import twl

def find_words(matrix, max_length, matrix_size, start_row=0, start_col=0):
    """
    Finds all words of length between 4 and max_length that can be made starting from the given position
    in the matrix, following the rules of Squaredle.

    Args:
        matrix (list): A 3x3 matrix of letters represented as a list of lists.
        start_row (int): The starting row index (0-based).
        start_col (int): The starting column index (0-based).
        max_length (int): The maximum length of words to find.

    Returns:
        list: A list of words that can be formed following the given rules.
    """
    def is_valid(row, col, word):
        """
        Helper function to check if a given position and word form a valid word
        according to the rules of Squaredle.

        Args:
        row (int): The current row index.
        col (int): The current column index.
        word (str): The current word being formed.

        Returns:
        bool: True if the word is valid, False otherwise.
        """
        # Check if the position is within the matrix boundaries
        if row < 0 or row >= matrix_size or col < 0 or col >= matrix_size:
            return False

        # Check if the current word length is within the desired range
        if len(word) < 4 or len(word) > max_length:
            return False

        return True

    def dfs(row, col, word):
        """
        Recursive depth-first search function to explore neighboring positions
        and form words according to the rules of Squaredle.

        Args:
            row (int): The current row index.
            col (int): The current column index.
            word (str): The current word being formed.

        Returns:
            None
        """
        nonlocal words

        # Base case: if the word length is greater than the maximum length, return
        if len(word) > max_length:
            return

        # Check if the current position and word form a valid word
        if is_valid(row, col, word) and twl.check(word.lower()):
            words.add(word)

        # Explore neighboring positions
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                new_row = row + dr
                new_col = col + dc
                # Skip the current position and out-of-bound positions
                if (new_row, new_col) != (row, col) and 0 <= new_row < matrix_size and 0 <= new_col < matrix_size:
                    # Skip already visited positions
                    if (new_row, new_col) not in visited:
                        visited.add((new_row, new_col))
                        dfs(new_row, new_col, word + matrix[new_row][new_col])
                        visited.remove((new_row, new_col))

    words = set()  # Set to store unique words
    visited = {(start_row, start_col)}  # Set to store visited positions

    # Iterate over all cells in the matrix to find words
    for row in range(matrix_size):
        for col in range(matrix_size):
            visited.add((row, col))
            dfs(row, col, matrix[row][col])
            visited.remove((row, col))

    words = sorted(words, key=lambda w: (len(w), w))
    
    #valid_words = [word for word in words if twl.check(word.lower())]

    return words

class GridGUI:
    def __init__(self, master):
        self.master = master
        master.title("Squaredle Solver")

        # create label and entry for grid size
        self.size_label = tk.Label(master, text="Grid Size:")
        self.size_label.grid(row=0, column=0)
        self.size_entry = tk.Entry(master)
        self.size_entry.grid(row=0, column=1)

        # create button to generate grid
        self.generate_button = tk.Button(master, text="Generate Grid", command=self.generate_grid)
        self.generate_button.grid(row=0, column=2)

        # create label and entry for max length
        self.max_len_label = tk.Label(master, text="Max Word Length:")
        self.max_len_label.grid(row=1, column=0)
        self.max_len_entry = tk.Entry(master)
        self.max_len_entry.grid(row=1, column=1)

        # create button to run function
        self.run_button = tk.Button(master, text="Solve", command=self.run_function)
        self.run_button.grid(row=1, column=2)

        # create empty list for grid of textboxes
        self.grid = []

        # create scrollable listbox for function output
        self.output_frame = tk.Frame(master)
        self.output_frame.grid(row=2, column=0, columnspan=3)
        self.output_label = tk.Listbox(self.output_frame)
        self.output_label.pack(side="left", fill="both", expand=True)
        self.output_scrollbar = ttk.Scrollbar(self.output_frame, orient="vertical", command=self.output_label.yview)
        self.output_scrollbar.pack(side="right", fill="y")
        self.output_label.config(yscrollcommand=self.output_scrollbar.set)

    def generate_grid(self):
        # clear previous grid if it exists
        if self.grid:
            for row in self.grid:
                for box in row:
                    box.destroy()
        self.grid = []

        # get size of grid from entry
        size = int(self.size_entry.get())

        # create grid of textboxes
        for i in range(size):
            row = []
            for j in range(size):
                box = tk.Entry(self.master, width=5)
                box.grid(row=i+3, column=j, padx=15, pady=5)
                row.append(box)
            self.grid.append(row)
        
        for i in range(size):
            self.master.columnconfigure(i, weight=1)

    def run_function(self):
        # get input for function from entry
        max_length = int(self.max_len_entry.get())

        # get characters from grid of textboxes
        characters = []
        for row in self.grid:
            row_characters = []
            for box in row:
                row_characters.append(box.get())
            characters.append(row_characters)

        # run function with input and characters
        function_output = self.solve(max_length, characters)
        
        for idx, wd in enumerate(function_output):
            self.output_label.insert(idx, wd)

        # update output label with function output
        #self.output_label.config(text=function_output)

    def solve(self, max_len, characters):
        # replace this with your actual function code
        output = find_words(characters, max_len, int(self.size_entry.get()))
        return output

root = tk.Tk()
my_gui = GridGUI(root)
root.mainloop()